# Parsing Json Messages

In this exercises we will send a stream of json object. You can look at the code to understand the structure.
You will create a spark streaming listener that will display the sum of all items per route.

In [ ]:
import com.amazonaws.services.kinesis.model.PutRecordRequest
import com.amazonaws.services.kinesis.AmazonKinesisClientBuilder
import com.amazonaws.auth.{AWSStaticCredentialsProvider, BasicAWSCredentials}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import collection.JavaConverters._

In [ ]:
val awsAccessKeyId = "YOUR ACCESS KEY ID"
val awsSecretKey = "YOUR SECRET KEY"
val kinesisStreamName = "YOUR STREAM NAME"
val kinesisRegion = "YOUR REGION"

In [ ]:
val kinesis = spark.readStream
  .format("kinesis")
  .option("streamName", kinesisStreamName)
  .option("region", kinesisRegion)
  .option("initialPosition", "TRIM_HORIZON")
  .option("checkpointLocation", "s3://output-databricks/checkpoint/")
  .option("awsAccessKey", awsAccessKeyId)
  .option("awsSecretKey", awsSecretKey)
  .load()


val schema = StructType(Seq(
  StructField("items", ArrayType(StringType)),
  StructField("from", StringType),
  StructField("to", StringType),
))

val result = kinesis.selectExpr("CAST(data as STRING) as json")
  .withColumn("order", from_json($"json", schema)) // we need to parse the json into a temporary column
  .withColumn("item_count", size(expr("order.items"))) // we extract the json fields.
  .withColumn("from", expr("order.from"))
  .withColumn("to", expr("order.to"))
  .drop("json") // we drop the temporary column
  .drop("order") // we drop the temporary column
  .groupBy("from", "to").agg(sum($"item_count"))

In [ ]:
display(result)

In [ ]:
import scala.util.Random
import java.lang.reflect.{Type, ParameterizedType}
import com.fasterxml.jackson.databind.ObjectMapper
import com.fasterxml.jackson.module.scala.DefaultScalaModule
import com.fasterxml.jackson.annotation.JsonProperty;
import com.fasterxml.jackson.core.`type`.TypeReference;
import java.nio.ByteBuffer

In [ ]:
case class Order(from: String, to: String, items: Seq[String])

def getRandomElement[A](seq: Seq[A]): A = {
  val r = scala.util.Random
  seq(r.nextInt(seq.length))
}
    
def selectNElementFromList[A](maxElements: Int, list: Seq[A]): Seq[A] = {
  val r = scala.util.Random
  (0 until r.nextInt(maxElements) + 1).map(idx => list(r.nextInt(list.length))).toSeq
}

// As in the previous exercice we initialize the kinesis client
val kinesisClient = AmazonKinesisClientBuilder.standard()
  .withRegion(kinesisRegion)
  .withCredentials(new AWSStaticCredentialsProvider(new BasicAWSCredentials(awsAccessKeyId, awsSecretKey)))
  .build()

println(s"Putting words onto stream $kinesisStreamName")
var lastSequenceNumber: String = null

val SOURCES = Seq("Paris", "Lyon", "Marseille", "Bordeau")
val DESTINATIONS = Seq("Berlin", "Madrid", "Rome", "London")
val ITEMS = Seq("Adidas Kampung","Ballet shoe","Pointe shoe","Bast shoe","Blucher shoe","Boat shoe","Brogan","Brogue shoe","Brothel creeper","Bucks","Cantabrian albarcas","Chelsea boot","Chopine","Chukka boot","Climbing shoe","Clog","Court shoe","Cross country running shoe","Derby shoe","Desert Boot","Diabetic shoe","Dress shoe","Duckbill shoe","Driving moccasins","Earth shoe","Elevator shoes","Espadrille","Fashion boot","Galesh","Geta","Giveh","High-heeled")

// ObjectMapper is a helper to tranform objects into a json string.
val mapper = new ObjectMapper()
mapper.registerModule(DefaultScalaModule)

for (i <- 0 to 10) {
  val time = System.currentTimeMillis
  
  val data = mapper.writeValueAsString(Order(getRandomElement(SOURCES), getRandomElement(DESTINATIONS), selectNElementFromList(5, ITEMS)))
  val request = new PutRecordRequest()
      .withStreamName(kinesisStreamName)
      .withPartitionKey("some-key")
      .withData(ByteBuffer.wrap(data.getBytes()))
  if (lastSequenceNumber != null) {
    request.setSequenceNumberForOrdering(lastSequenceNumber)
  }    
  val result = kinesisClient.putRecord(request)
  lastSequenceNumber = result.getSequenceNumber()

  Thread.sleep(math.max(10000 - (System.currentTimeMillis - time), 0)) // loop around every ~10 seconds 
}